In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()

train_x = train_x[:50]
train_y = train_y[:50]

test_x = test_x[:50]
test_y = test_y[:50]

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
train_x = train_x.astype("float32") / 255.0
test_x = test_x.astype("float32") / 255.0

train_x = train_x.reshape(-1, 28, 28, 1)
test_x = test_x.reshape(-1, 28, 28, 1)

x_train, x_val, y_train, y_val = train_test_split(
    train_x, train_y, test_size=0.2, random_state=101
)

In [4]:
def build_cnn():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [5]:
single_net = build_cnn()

single_net.fit(
    x_train, y_train,
    epochs=3,
    validation_data=(x_val, y_val),
    verbose=1
)

val_pred_single = np.argmax(single_net.predict(x_val), axis=1)
test_pred_single = np.argmax(single_net.predict(test_x), axis=1)

baseline_val_acc = accuracy_score(y_val, val_pred_single)
baseline_test_acc = accuracy_score(test_y, test_pred_single)

Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 443ms/step - accuracy: 0.0979 - loss: 2.3047 - val_accuracy: 0.3000 - val_loss: 2.1854
Epoch 2/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - accuracy: 0.3854 - loss: 2.1175 - val_accuracy: 0.3000 - val_loss: 2.1057
Epoch 3/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.6188 - loss: 1.9661 - val_accuracy: 0.6000 - val_loss: 2.0193
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step 


In [6]:
ensemble_members = []
ensemble_size = 5

for n in range(ensemble_size):
    print(f"Training model {n+1}/{ensemble_size}")

    sample_ids = np.random.choice(len(x_train), size=len(x_train), replace=True)

    boot_x = x_train[sample_ids]
    boot_y = y_train[sample_ids]

    net = build_cnn()

    net.fit(
        boot_x, boot_y,
        epochs=3,
        verbose=1
    )

    ensemble_members.append(net)

Training model 1/5
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.0333 - loss: 2.3210  
Epoch 2/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.4354 - loss: 2.0983
Epoch 3/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.4625 - loss: 1.9569
Training model 2/5
Epoch 1/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.0979 - loss: 2.2995
Epoch 2/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 2.0912
Epoch 3/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.4792 - loss: 1.9267
Training model 3/5
Epoch 1/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.0708 - loss: 2.3101 
Epoch 2/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.2667 - loss: 2.0949
Epoch 3/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5062 - loss: 1.9384
Training model 4/5
Epoch 1/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.0604 - loss: 2.3307 
Epoch 2/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4188 - loss: 2.0675
Epoch 3/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

In [7]:
def get_ensemble_prediction(models, data):
    prob_collect = [m.predict(data) for m in models]

    prob_mean = np.mean(prob_collect, axis=0)

    final_output = np.argmax(prob_mean, axis=1)
    return final_output

In [8]:
ensemble_val_pred = get_ensemble_prediction(ensemble_members, x_val)
ensemble_val_acc = accuracy_score(y_val, ensemble_val_pred)

ensemble_test_pred = get_ensemble_prediction(ensemble_members, test_x)
ensemble_test_acc = accuracy_score(test_y, ensemble_test_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step 


In [9]:
print("=============================================")
print("            MODEL PERFORMANCE SUMMARY        ")
print("=============================================")

print(f"Single CNN - Validation Accuracy : {baseline_val_acc:.4f}")
print(f"Single CNN - Test Accuracy       : {baseline_test_acc:.4f}")

print("---------------------------------------------")

print(f"Ensemble CNN - Validation Acc    : {ensemble_val_acc:.4f}")
print(f"Ensemble CNN - Test Acc          : {ensemble_test_acc:.4f}")
print("=============================================")

            MODEL PERFORMANCE SUMMARY        
Single CNN - Validation Accuracy : 0.6000
Single CNN - Test Accuracy       : 0.3600
---------------------------------------------
Ensemble CNN - Validation Acc    : 0.4000
Ensemble CNN - Test Acc          : 0.3200
